In [4]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.multioutput import MultiOutputRegressor

# Load Datasets

In [2]:
def load_data(file_path):
    A = np.loadtxt(file_path)
    X = A[:, :9]    # Input features
    y = A[:, 9:]    # Output labels
    return X, y

In [8]:
X_multi, y_multi = load_data('data/tictac_multi.txt')

# Linear Regression

In [16]:
def lr_train(X, y):
    # Split into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

    # Define and train model
    base_classifier = LinearRegression()
    model = MultiOutputRegressor(base_classifier)
    model.fit(X_train, y_train)

    # Get accuracy
    accuracy = cross_val_score(model, X_train, y_train, cv=10, scoring="accuracy")
    mean_accuracy = np.mean(accuracy)

    # Get confusion matrix
    y_pred = model.predict(X_test)
    print(y_pred.shape)
    print(y_test.shape)
    #confusion_mtrx = multilabel_confusion_matrix(y_test, y_pred)

    return mean_accuracy#, confusion_mtrx

In [11]:
def print_results(accuracy, confusion_mtrx, dataset_name):
    print(f"Performance of Linear Regression on {dataset_name}:")
    print("Accuracy = ", accuracy)
    print("Confusion Matrix:")

    for i, matrix in enumerate(confusion_mtrx):
        print(f"Confusion Matrix for Label {i}:")

        # Normalize confusion matrix
        row_sums = matrix.sum(axis=1, keepdims=True)
        matrix = matrix / row_sums
        print(matrix.round(decimals=3))
        
    print("\n")

## Multi Dataset

In [18]:
#accuracy_multi, confusion_mtrx_multi = lr_train(X_multi, y_multi)
accuracy_multi = lr_train(X_multi, y_multi)
#print_results(accuracy_multi, [[]], "Multi Dataset")

c:\Users\helar\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:842: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\helar\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 136, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\helar\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\helar\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 211, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\helar\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py", line 220, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  

(1311, 9)
(1311, 9)


c:\Users\helar\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:842: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\helar\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 136, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\helar\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\helar\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 211, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\helar\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py", line 220, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  